In [1]:
# from tkinter import image_names
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import os
import pathlib
import random
import matplotlib.pyplot as plt
import numpy as np


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'



def read_data(path):
    path_root = pathlib.Path(path)
    # print(path_root)
    # for item in path_root.iterdir():
    #     print(item)
    image_paths = list(path_root.glob('*/*'))
    #print(image_paths)
    image_paths = [str(path) for path in image_paths]
    #print(image_paths)
    random.shuffle(image_paths)
    image_count = len(image_paths)
    # print(image_count)
    # print(image_paths[:10])

    label_names = sorted(item.name for item in path_root.glob('*/') if item.is_dir())
    #print(label_names)
    label_name_index = dict((name, index) for index, name in enumerate(label_names))
    #print(label_name_index)
    image_labels = [label_name_index[pathlib.Path(path).parent.name] for path in image_paths]
    print(image_labels)
    #print("First 10 labels indices: ", image_labels[:10])
    return image_paths,image_labels,image_count


def binaryMask(frame,x0,y0,width,height):
    cv2.rectangle(frame,(x0,y0),(x0+width,y0+height),(0,255,0))
    roi = frame[y0:y0+height,x0:x0+width]
    #cv2.imshow("roi",roi)#读取roi文件
    res = skinMask(roi)
    #cv2.imshow("res",res)
    return res


def skinMask(roi):
    #rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)  # 转换到RGB空间
    CrCb=cv2.cvtColor(roi, COLOR_BGR2YCrCb)
    (Y,Cr,Cb) = cv2.split(CrCb)  # 获取图像每个像素点的RGB的值，即将一个二维矩阵拆成三个二维矩阵
    skin = np.zeros(Cr.shape, dtype=np.uint8)  # 掩膜
    (x, y) = Cb.shape  # 获取图像的像素点的坐标范围
    for i in range(0, x):
        for j in range(0, y):
            # 判断条件，不在肤色范围内则将掩膜设为黑色，即255
            # if (abs(R[i][j] - G[i][j]) > 15) and (R[i][j] > G[i][j]) and (R[i][j] > B[i][j]):
            #     if (R[i][j] > 95) and (G[i][j] > 40) and (B[i][j] > 20) \
            #             and (max(R[i][j], G[i][j], B[i][j]) - min(R[i][j], G[i][j], B[i][j]) > 15):
            #         skin[i][j] = 255
            #     elif (R[i][j] > 220) and (G[i][j] > 210) and (B[i][j] > 170):
            #         skin[i][j] = 255
            if (Cr[i][j] > 133) and (Cr[i][j] < 170) and (Cb[i][j] > 77) and (Cb[i][j] < 127):
                skin[i][j] = 255
            else:
                skin[i][j] = 0
    res = cv2.bitwise_and(roi, roi, mask=skin)  # 图像与运算
    return res


def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [100, 100])
    image /= 255.0  # normalize to [0,1] range
    # image = tf.reshape(image,[100*100*3])
    return image

def load_and_preprocess_image(path,label):
    image = tf.io.read_file(path)
    return preprocess_image(image),label

def creat_dataset(image_paths,image_labels,bitch_size):
    db = tf.data.Dataset.from_tensor_slices((image_paths, image_labels))
    #print("db:")
    
    dataset = db.map(load_and_preprocess_image).batch(bitch_size)  
    # for i in dataset:
    #     print(i)  
    return dataset


def train_model(train_data,test_data):
    #构建模型
    network = keras.Sequential([
            keras.layers.Conv2D(32,kernel_size=[5,5],padding="same",activation=tf.nn.relu),
            keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
            keras.layers.Conv2D(64,kernel_size=[3,3],padding="same",activation=tf.nn.relu),
            keras.layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),
            keras.layers.Conv2D(64,kernel_size=[3,3],padding="same",activation=tf.nn.relu),
            keras.layers.Flatten(),
            keras.layers.Dense(512,activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(128,activation='relu'),
            keras.layers.Dense(4)])
    network.build(input_shape=(None,100,100,3))
    network.summary()

    network.compile(optimizer=optimizers.SGD(lr=0.001),
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
    )
    #模型训练
    network.fit(train_data, epochs = 35,validation_data=test_data,validation_freq=2)  
    network.evaluate(test_data)

    tf.saved_model.save(network,'./model')
    print("保存模型成功")



    # Convert Keras model to ConcreteFunction
    full_model = tf.function(lambda x: network(x))
    full_model = full_model.get_concrete_function(
        tf.TensorSpec(network.inputs[0].shape, network.inputs[0].dtype))

    # Get frozen ConcreteFunction
    frozen_func = convert_variables_to_constants_v2(full_model)
    frozen_func.graph.as_graph_def()

    layers = [op.name for op in frozen_func.graph.get_operations()]
    print("-" * 50)
    print("Frozen model layers: ")
    for layer in layers:
        print(layer)

    print("-" * 50)
    print("Frozen model inputs: ")
    print(frozen_func.inputs)
    print("Frozen model outputs: ")
    print(frozen_func.outputs)                                                                                               

    # Save frozen graph from frozen ConcreteFunction to hard drive
    tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
            logdir="./frozen_model",
            name="frozen_graph.pb",
            as_text=False)
    print("模型转换完成，训练结束")


if  __name__ == "__main__":
    print(tf.__version__)
    train_path = 'F:\\dataSet\\dataSetPro'
    test_path = 'F:\\dataSet\\testdata' 
    image_paths,image_labels,_ = read_data(train_path)
    train_data = creat_dataset(image_paths,image_labels,16)
    image_paths,image_labels,_ = read_data(test_path)
    test_data = creat_dataset(image_paths,image_labels,16)
    train_model(train_data,test_data)
    


2.3.0
[2, 0, 2, 3, 3, 2, 1, 2, 1, 0, 3, 2, 2, 3, 1, 2, 2, 3, 2, 3, 2, 2, 2, 0, 1, 2, 3, 3, 0, 0, 2, 0, 3, 2, 0, 2, 2, 2, 3, 2, 2, 0, 0, 2, 2, 0, 0, 0, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 2, 3, 3, 0, 0, 1, 1, 2, 2, 1, 0, 1, 0, 2, 1, 1, 0, 1, 0, 2, 0, 3, 0, 3, 2, 1, 0, 3, 2, 3, 2, 2, 3, 2, 2, 0, 2, 2, 2, 0, 0, 3, 1, 2, 2, 0, 0, 1, 2, 0, 3, 3, 1, 1, 2, 3, 2, 1, 3, 3, 1, 0, 1, 3, 0, 2, 1, 1, 2, 0, 3, 2, 3, 3, 0, 3, 1, 3, 0, 1, 1, 1, 0, 0, 3, 1, 0, 0, 3, 2, 0, 3, 0, 2, 1, 1, 1, 3, 3, 3, 3, 0, 3, 1, 1, 0, 3, 0, 0, 0, 1, 0, 2, 2, 1, 2, 1, 2, 3, 1, 2, 0, 1, 0, 0, 1, 3, 2, 3, 1, 0, 0, 0, 3, 0, 2, 2, 0, 0, 1, 2, 3, 0, 0, 1, 0, 3, 0, 3, 1, 1, 3, 0, 0, 2, 3, 1, 1, 2, 1, 0, 2, 0, 0, 1, 0, 3, 1, 3, 2, 3, 2, 0, 2, 1, 0, 0, 2, 0, 0, 3, 1, 0, 3, 3, 1, 3, 1, 3, 1, 2, 0, 0, 2, 0, 2, 1, 3, 3, 1, 2, 0, 3, 0, 2, 0, 2, 3, 1, 3, 3, 1, 0, 1, 3, 2, 2, 1, 3, 2, 3, 2, 0, 3, 2, 1, 1, 1, 0, 1, 1, 3, 3, 2, 3, 1, 1, 0, 0, 1, 2, 1, 0, 3, 2, 1, 3, 0, 3, 2, 3, 0, 2, 2, 2, 1, 3, 3, 3, 3, 2, 2, 3, 1, 3, 3, 1, 1, 0, 2, 1, 1, 

In [ ]:
tf.compat.v1.enable_eager_execution()
[9, 9, 2, 1, 2, 2, 7, 8, 4, 4]